In [14]:
import json
import pandas as pd

SPOTIFY_DATA_PATH = 'StreamingHistoryMartyna.json'
GOOGLE_MAPS_DECEMBER_DATA_PATH = '2023_DECEMBER.json'
GOOGLE_MAPS_JANUARY_DATA_PATH = '2024_JANURARY.json'
DROP_EMPTY_SONGS = False
TRESHOLD_MS_PLAYED= 30000


f=open(GOOGLE_MAPS_DECEMBER_DATA_PATH)
google_maps_data=json.load(f)
f.close()
f=open(SPOTIFY_DATA_PATH)
spotify=json.load(f)
f.close()
dfSpotify=pd.DataFrame(spotify)
dfSpotify['endTime'] = pd.to_datetime(dfSpotify['endTime']).dt.tz_localize(None)
dfSpotify = dfSpotify[dfSpotify['msPlayed'] > TRESHOLD_MS_PLAYED]




def find_songs(start_time, end_time):
    start_time = pd.to_datetime(start_time).tz_convert(None).strftime('%Y-%m-%d %H:%M:%S')

    end_time = pd.to_datetime(end_time).tz_convert(None).strftime('%Y-%m-%d %H:%M:%S')

    mask = (dfSpotify['endTime'] >= start_time) & (dfSpotify['endTime'] <= end_time)
    relevant_songs = dfSpotify[mask]
    return relevant_songs[['trackName', 'artistName','msPlayed']].to_dict('records')

activities_routes = []

for timeline_object in google_maps_data['timelineObjects']:
    if "activitySegment" in timeline_object:
        activity_segment_json = timeline_object["activitySegment"]

        startTimestamp = pd.Timestamp(activity_segment_json["duration"]["startTimestamp"])
        endTimestamp = pd.Timestamp(activity_segment_json["duration"]["endTimestamp"])

        route = []

        if "waypointPath" in activity_segment_json:
            waypoints = activity_segment_json["waypointPath"]["waypoints"]
            for waypoint in waypoints:
                lat = waypoint["latE7"] / 1e7
                lon = waypoint["lngE7"] / 1e7
                route.append((lat, lon))

        elif "simplifiedRawPath" in activity_segment_json:
            rawPath = activity_segment_json["simplifiedRawPath"]["points"]
            for point in rawPath:
                lat = point["latE7"] / 1e7
                lon = point["lngE7"] / 1e7
                route.append((lat, lon))

        if route:
            songs = find_songs(startTimestamp, endTimestamp)
            activities_routes.append({
                "startTimestamp": startTimestamp,
                "endTimestamp": endTimestamp,
                "activityType": activity_segment_json["activityType"],
                "route": route,
                "songs": songs
            })



f=open('2024_JANUARY.json')
google_maps_data=json.load(f)
f.close()



for timeline_object in google_maps_data['timelineObjects']:
    if "activitySegment" in timeline_object:
        activity_segment_json = timeline_object["activitySegment"]

        startTimestamp = pd.Timestamp(activity_segment_json["duration"]["startTimestamp"])
        endTimestamp = pd.Timestamp(activity_segment_json["duration"]["endTimestamp"])

        route = []

        if "waypointPath" in activity_segment_json:
            waypoints = activity_segment_json["waypointPath"]["waypoints"]
            for waypoint in waypoints:
                lat = waypoint["latE7"] / 1e7
                lon = waypoint["lngE7"] / 1e7
                route.append((lat, lon))

        elif "simplifiedRawPath" in activity_segment_json:
            rawPath = activity_segment_json["simplifiedRawPath"]["points"]
            for point in rawPath:
                lat = point["latE7"] / 1e7
                lon = point["lngE7"] / 1e7
                route.append((lat, lon))

        if route:
            songs = find_songs(startTimestamp, endTimestamp)
            activities_routes.append({
                "startTimestamp": startTimestamp,
                "endTimestamp": endTimestamp,
                "activityType": activity_segment_json["activityType"],
                "route": route,
                "songs": songs
            })


for timeline_object in google_maps_data['timelineObjects']:
    if "placeVisit" in timeline_object:
        place_visit_json = timeline_object["placeVisit"]

        location = place_visit_json["location"]
        address = location.get("address", "Unknown")
        name = location.get("name", "Unknown")
        
        latitude = location.get("latitudeE7", 0) / 1e7
        longitude = location.get("longitudeE7", 0) / 1e7

        startTimestamp = pd.Timestamp(place_visit_json["duration"]["startTimestamp"])
        endTimestamp = pd.Timestamp(place_visit_json["duration"]["endTimestamp"])

        songs = find_songs(startTimestamp, endTimestamp)
        activities_routes.append({
            "startTimestamp": startTimestamp,
            "endTimestamp": endTimestamp,
            "startAddress": address,
            "startName": name,
            "endAddress": address, 
            "endName": name,       
            "songs": songs,
            "latitude": latitude,  
            "longitude": longitude
        })

routes = pd.DataFrame(activities_routes)
if DROP_EMPTY_SONGS:
    routes = routes[routes['songs'].str.len() > 0]

routes.to_csv('routes_spotify_martyna.csv')
